## 1. 임포트

In [3]:
import pymongo
import requests
from bs4 import BeautifulSoup as BS
import re

## 2. 기반작업 - post로 html 가져오기 및 각 배우들 데이터 가져오기

In [39]:
# 몽고디비 연결
conn = pymongo.MongoClient()
actor_db = conn.actor_db
actor_collection = actor_db.actor_collection
final_list = list()

#크롤링 작업 - 데이터 수집 
url = 'http://www.cine21.com/rank/person/content'
page =0


while True:
    page+=1
    form_data = {'section': 'actor','period_start': '2020-06','gender': 'all','page': page}
    res = requests.post(url, data = form_data)
    soup = BS(res.content, 'lxml')
    if page==2:                
        #len(soup.select('li.people_li div.name'))==0:
        break
    else:
        actors = soup.select('li.people_li')
        for actor in actors:
            data_dict = dict()
            actor_name = actor.select_one('div.name')
            actor_link ='http://www.cine21.com'+actor_name.a['href']
            response_actor = requests.get(actor_link)
            soup = BS(response_actor.content, 'lxml')
            infos = soup.select('div.default_info_area li')
            
            
            data_dict['배우이름'] =re.sub('\(.+\)','',actor_name.get_text())
            data_dict['흥행지수']=int(actor.select_one('ul.num_info strong').get_text().replace(',',''))
            movies = actor.select('ul.mov_list li')
            movie_list = list()
            for movie in movies:
                movie_list.append(movie.get_text().strip())
            data_dict['출연영화']=movie_list
            data_dict['순위']=int(actor.select_one('span.grade').get_text())
            #print('<'+re.sub('\(.+\)','',actor.get_text())+'>')
            
            for info in infos:
                detail = re.sub('<span.+n>','',str(info))
                detail = re.sub('<.*?>','',detail)
                data_dict[info.span.get_text()]=detail
                
            try:
                homepages = data_dict['홈페이지']
                homepages = homepages.split('\n')
                for homepage in homepages:
                    homepages.remove('')
                homepages = list(map(lambda x: x.strip(),homepages))
                data_dict['홈페이지']=homepages
            except KeyError:
                data_dict['홈페이지']='홈페이지 없음'
                
            
            print(data_dict)
            final_list.append(data_dict)   

#print(final_list)
#actor_collection.insert_many(final_list)

{'배우이름': '황정민', '흥행지수': 85080, '출연영화': ['다만 악에서 구하소서', '신세계', '국제시장'], '순위': 1, '직업': '배우', '생년월일': '1970-09-01', '성별': '남', '신장/체중': '180cm, 75kg', '학교': '서울예술대학 연극과 졸업', '특기': '농구, 악기연주', '소속사': '예당엔터테인먼트', '홈페이지': '홈페이지 없음'}
{'배우이름': '이정재', '흥행지수': 56738, '출연영화': ['다만 악에서 구하소서', '신과 함께-죄와 벌', '태양은 없다', '시월애', '신과 함께-인과 연', '신세계'], '순위': 2, '직업': '배우', '성별': '남', '홈페이지': ['http://www.artistcompany.co.kr/portfolio-item/lee-jung-jae/'], '소속사': '아티스트컴퍼니'}
{'배우이름': '유아인', '흥행지수': 46364, '출연영화': ['#살아있다', '소리도 없이', '버닝', '국가부도의 날', '사도'], '순위': 3, '다른 이름': '엄홍식', '직업': '배우', '생년월일': '1986-10-06', '성별': '남', '홈페이지': ['https://www.instagram.com/hongsick/', 'https://twitter.com/seeksik'], '신장/체중': '178cm, 60kg', '학교': '단국대학교 연극', '취미': '피아노 연주, 인터넷 게임, 영화, 음악감상', '특기': '스노우보드, 수영'}
{'배우이름': '강동원', '흥행지수': 31818, '출연영화': ['반도', '1987'], '순위': 4, '직업': '배우', '생년월일': '1981-01-18', '성별': '남', '홈페이지': ['http://www.gangdongwon.com'], '신장/체중': '186cm, 68kg', '학교': '한양대 - 상명대학교대학원 연극영화 (석사과정 재학)', '

In [27]:
data_dict={'배우이름': '이정현', '흥행지수': 28953, '출연영화': ['죽지않는 인간들의 밤', '반도', '명량'], '순위': 5, '직업': '배우', '생년월일': '1980-02-07', '성별': '여', '홈페이지': '\nhttps://www.instagram.com/mermaidleejh/\nhttps://twitter.com/mermaidjh\n', '신장/체중': '158cm, 38kg', '학교': '중앙대학교 연극영화', '취미': '바비 인형 수집, 피아노 연주', '특기': '별관찰, 시나리오 쓰기', '소속사': '에이바 앤 엔터테인먼트'}
homepages = data_dict['홈페이지']
homepages = homepages.split('\n')
for homepage in homepages:
    homepages.remove('')
print(homepages)

['https://www.instagram.com/mermaidleejh/', 'https://twitter.com/mermaidjh']


In [32]:
data_dict['d']

KeyError: 'd'

In [26]:
a =data_dict['홈페이지']
homepage = a.split('\n')
for h in map(lambda x: x.strip(),homepage):
    if len(h)==0:
        homepage.remove('')
    
print(homepage)

['https://www.facebook.com/shs831', ' https://www.instagram.com/shinhs831/']


In [14]:
' https://www.instagram.com/shinhs831/'.strip()

'https://www.instagram.com/shinhs831/'

In [19]:
a = ['https://www.facebook.com/shs831', ' https://www.instagram.com/shinhs831/']
b = list(map(lambda x: x.strip(),a))
print(b)

['https://www.facebook.com/shs831', 'https://www.instagram.com/shinhs831/']


In [11]:
home=a.split('\n')
for a i

['https://www.facebook.com/shs831', ' https://www.instagram.com/shinhs831/', '']


In [150]:
['d','a'].remove('a')

In [151]:
a=['d','a']
a.remove('a')
print(a)

['d']


In [108]:
res = requests.post(url, data = {'section': 'actor','period_start': '2020-06','gender': 'all','page': 1})
soup = BS(res.content,'lxml')
actors = soup.select('li.people_li div.name')
print(actors)

[<div class="name"><a href="/db/person/info/?person_id=14268">황정민(3편)</a></div>, <div class="name"><a href="/db/person/info/?person_id=1075">이정재(7편)</a></div>, <div class="name"><a href="/db/person/info/?person_id=57657">유아인(5편)</a></div>, <div class="name"><a href="/db/person/info/?person_id=18040">강동원(2편)</a></div>, <div class="name"><a href="/db/person/info/?person_id=2760">이정현(3편)</a></div>, <div class="name"><a href="/db/person/info/?person_id=74826">박정민(5편)</a></div>, <div class="name"><a href="/db/person/info/?person_id=27826">성동일(2편)</a></div>]


In [43]:
import re
p = re.compile('\(\w+\)')

In [137]:
final_list

[{'배우이름': '황정민',
  '흥행지수': 85103,
  '출연영화': ['다만 악에서 구하소서', '신세계', '국제시장'],
  '순위': 1,
  '직업': '배우',
  '생년월일': '1970-09-01',
  '성별': '남',
  '신장/체중': '180cm, 75kg',
  '학교': '서울예술대학 연극과 졸업',
  '특기': '농구, 악기연주',
  '소속사': '예당엔터테인먼트'},
 {'배우이름': '이정재',
  '흥행지수': 56753,
  '출연영화': ['다만 악에서 구하소서', '신과 함께-죄와 벌', '태양은 없다', '시월애', '신과 함께-인과 연', '신세계'],
  '순위': 2,
  '직업': '배우',
  '성별': '남',
  '홈페이지': ' http://www.artistcompany.co.kr/portfolio-item/lee-jung-jae/ ',
  '소속사': '아티스트컴퍼니'},
 {'배우이름': '유아인',
  '흥행지수': 46376,
  '출연영화': ['#살아있다', '소리도 없이', '버닝', '국가부도의 날', '사도'],
  '순위': 3,
  '다른 이름': '엄홍식',
  '직업': '배우',
  '생년월일': '1986-10-06',
  '성별': '남',
  '홈페이지': ' https://www.instagram.com/hongsick/ https://twitter.com/seeksik ',
  '신장/체중': '178cm, 60kg',
  '학교': '단국대학교 연극',
  '취미': '피아노 연주, 인터넷 게임, 영화, 음악감상',
  '특기': '스노우보드, 수영'},
 {'배우이름': '강동원',
  '흥행지수': 31826,
  '출연영화': ['반도', '1987'],
  '순위': 4,
  '직업': '배우',
  '생년월일': '1981-01-18',
  '성별': '남',
  '홈페이지': ' http://www.gangdongwon.com ',
  '신장/체

In [47]:
re.sub(p,'',actor.get_text())

'성동일'

In [82]:
for actor in actors:
    actor_link ='http://www.cine21.com'+actor.a['href']
    response_actor = requests.get(actor_link)
    soup = BS(response_actor.content, 'lxml')
    infos = soup.select('div.default_info_area li')
    
    print('<'+re.sub(p,'',actor.get_text())+'>')
    
    for info in infos:
        detail = re.sub('<span.+n>','',str(info))
        detail = re.sub('<.*?>','',detail)
        print(info.span.get_text(),':',detail)
        
    print('--------------------------------')

<황정민>
직업 : 배우
생년월일 : 1970-09-01
성별 : 남
신장/체중 : 180cm, 75kg
학교 : 서울예술대학 연극과 졸업
특기 : 농구, 악기연주
소속사 : 예당엔터테인먼트
--------------------------------
<이정재>
직업 : 배우
성별 : 남
홈페이지 : 
http://www.artistcompany.co.kr/portfolio-item/lee-jung-jae/

소속사 : 아티스트컴퍼니
--------------------------------
<유아인>
다른 이름 : 엄홍식
직업 : 배우
생년월일 : 1986-10-06
성별 : 남
홈페이지 : 
https://www.instagram.com/hongsick/
https://twitter.com/seeksik

신장/체중 : 178cm, 60kg
학교 : 단국대학교 연극
취미 : 피아노 연주, 인터넷 게임, 영화, 음악감상
특기 : 스노우보드, 수영
--------------------------------
<강동원>
직업 : 배우
생년월일 : 1981-01-18
성별 : 남
홈페이지 : 
http://www.gangdongwon.com

신장/체중 : 186cm, 68kg
학교 : 한양대 - 상명대학교대학원 연극영화 (석사과정 재학)
취미 : 음악감상, 웹서핑, 운동, 신발 모으기
특기 : 축구
--------------------------------
<이정현>
직업 : 배우
생년월일 : 1980-02-07
성별 : 여
홈페이지 : 
https://www.instagram.com/mermaidleejh/
https://twitter.com/mermaidjh

신장/체중 : 158cm, 38kg
학교 : 중앙대학교 연극영화
취미 : 바비 인형 수집, 피아노 연주
특기 : 별관찰, 시나리오 쓰기
소속사 : 에이바 앤 엔터테인먼트
--------------------------------
<박정민>
직업 : 배우
생년월일 : 1987-02-25
성별 : 남
신장/체